In [3]:
# Import the MySQL connector library to interact with the MySQL database
import mysql.connector

# Function to create a new database
def create_database():
    try:
        # Establish a connection to the MySQL server without specifying a database
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="1234567890"
        )
        cursor = conn.cursor()
        # Drop the database if it already exists to start fresh
        cursor.execute("DROP DATABASE IF EXISTS crime_data_db")
        # Create a new database named 'crime_data_db'
        cursor.execute("CREATE DATABASE crime_data_db")
        print("Database created successfully........")
    except mysql.connector.Error as err:
        # Print any errors that occur during the process
        print(f"Error: {err}")
    finally:
        # Ensure the connection is closed properly
        if conn.is_connected():
            cursor.close()
            conn.close()

# Function to create a table within the database
def create_table():
    try:
        # Establish a connection to the MySQL server and specify the database to use
        conn = mysql.connector.connect(
            host="localhost",
            database="crime_data_db",
            user="root",
            password="1234567890"
        )
        cursor = conn.cursor()
        # Execute the SQL command to create a table named 'crime_data'
        cursor.execute('''
            CREATE TABLE crime_data (
                incident_date DATE,          -- Column for the date of the incident
                state VARCHAR(100),          -- Column for the state where the incident occurred
                city VARCHAR(100),           -- Column for the city where the incident occurred
                address VARCHAR(255),        -- Column for the address of the incident
                victims_killed INT,          -- Column for the number of victims killed
                victims_injured INT,         -- Column for the number of victims injured
                suspects_killed INT,         -- Column for the number of suspects killed
                suspects_injured INT,        -- Column for the number of suspects injured
                suspects_arrested INT        -- Column for the number of suspects arrested
            )
        ''')
        # Commit the changes to the database
        conn.commit()
        print("Table created successfully........")
    except mysql.connector.Error as err:
        # Print any errors that occur during the process
        print(f"Error: {err}")
    finally:
        # Ensure the connection is closed properly
        if conn.is_connected():
            cursor.close()
            conn.close()

# Call the functions to create the database and the table
create_database()
create_table()


Database created successfully........
Table created successfully........


In [4]:
# Import necessary libraries
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import mysql.connector
import time

# Function to create and return a connection to the MySQL database
def get_db_connection():
    conn = mysql.connector.connect(
        host="localhost",  # Database host
        database="crime_data_db",  # Database name
        user="root",  # Database user
        password="1234567890"  # Database password
    )
    return conn

# Function to configure and return a Selenium WebDriver for Chrome
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)
    return driver

# Function to navigate to a URL using Selenium and return the page's BeautifulSoup object
def scrape_data(driver, base_url):
    driver.get(base_url)
    time.sleep(5)  # Wait for the dynamic content to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

# Function to parse data from a BeautifulSoup object and insert it into the MySQL database
def parse_and_insert_data(soup, conn):
    cursor = conn.cursor()  # Create a cursor object using the connection
    data = []  # Initialize a list to store the data records
    table = soup.find('table', class_='sticky-enabled')  # Find the table with class 'sticky-enabled'
    if table:
        rows = table.find('tbody').find_all('tr')  # Find all rows in the table body
        for row in rows:
            cols = row.find_all('td')  # Find all columns in the row
            if cols:
                try:
                    # Convert the incident date from string to a datetime object and then format it for MySQL
                    date_str = cols[1].text.strip()
                    formatted_date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y-%m-%d')
                    # Prepare the record for insertion
                    record = (
                        formatted_date,
                        cols[2].text.strip(),  # State
                        cols[3].text.strip(),  # City or County
                        cols[4].text.strip(),  # Address
                        int(cols[5].text.strip()),  # Victims Killed
                        int(cols[6].text.strip()),  # Victims Injured
                        int(cols[7].text.strip()),  # Suspects Killed
                        int(cols[8].text.strip()),  # Suspects Injured
                        int(cols[9].text.strip()),  # Suspects Arrested
                    )
                    data.append(record)  # Add the record to the data list
                    # SQL query to insert the record into the database
                    query = '''INSERT INTO crime_data
                               (incident_date, state, city, address, victims_killed, victims_injured, suspects_killed, suspects_injured, suspects_arrested)
                               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                    cursor.execute(query, record)  # Execute the query with the record data
                    conn.commit()  # Commit the transaction
                    print("Inserted data:", record)  # Print the inserted data for verification
                except Exception as e:
                    print(f"Error inserting data: {e}")  # Print any error during insertion
    else:
        print("Table not found.")  # Print a message if the table isn't found
    cursor.close()  # Close the cursor

# Main function to orchestrate the scraping and insertion process
def main():
    driver = get_driver()  # Get a Selenium WebDriver
    conn = get_db_connection()  # Get a database connection
    for year in range(2014, 2024):  # Iterate through the years of interest
        page = 0  # Initialize page counter
        has_next_page = True  # Flag to control pagination
        while has_next_page:  # While there are more pages to scrape
            # Construct the URL based on the year and page
            if year <= 2015:
                url = f"https://www.gunviolencearchive.org/reports/mass-shootings/{year}?page={page}"
            else:
                url = f"https://www.gunviolencearchive.org/reports/mass-shooting?year={year}&page={page}"
            print(f"Scraping data for {year}, page {page}")  # Print the current scraping action
            soup = scrape_data(driver, url)  # Scrape the page
            parse_and_insert_data(soup, conn)  # Parse the scraped data and insert it into the database
            next_page_link = soup.find('a', title="Go to next page")  # Find the link to the next page
            has_next_page = bool(next_page_link)  # Update the flag based on the presence of the next page link
            page += 1  # Increment the page counter
    driver.quit()  # Quit the WebDriver
    conn.close()  # Close the database connection
    print("Data scraping and insertion completed.")  # Print completion message

if __name__ == "__main__":
    main()  # Execute the main function


Scraping data for 2014, page 0
Inserted data: ('2014-12-29', 'Louisiana', 'New Orleans', 'Poydras and Bolivar', 0, 4, 0, 0, 0)
Inserted data: ('2014-12-27', 'California', 'Los Angeles', '8800 block of South Figueroa Street', 1, 3, 0, 0, 0)
Inserted data: ('2014-12-27', 'California', 'Sacramento', '4000 block of May Street', 0, 4, 0, 0, 0)
Inserted data: ('2014-12-26', 'Illinois', 'East St. Louis', '2500 block of Summit Avenue', 1, 3, 0, 0, 0)
Inserted data: ('2014-12-24', 'Missouri', 'Saint Louis', '18th and Pine', 1, 3, 0, 0, 0)
Inserted data: ('2014-12-23', 'Kentucky', 'Winchester', '260 Oxford Dr', 1, 3, 0, 0, 2)
Inserted data: ('2014-12-22', 'Michigan', 'Detroit', 'Charlevoix and Philip', 1, 3, 0, 0, 0)
Inserted data: ('2014-12-22', 'New York', 'Webster', '191 Lake Rd', 3, 2, 1, 0, 0)
Inserted data: ('2014-12-22', 'Illinois', 'Chicago', '5700 block of South Green Street', 0, 5, 0, 0, 0)
Inserted data: ('2014-12-21', 'Florida', 'Sarasota', '4034 N Washington Blvd', 2, 2, 0, 0, 0)
In